In [ ]:
import numpy as np
import matplotlib.pyplot as plt

import pandas as pd

In [ ]:
class Plot:
    def __init__(self, file, names):        
        n_all = [pd.read_csv('stats/node_{}/{}'.format(node, file), sep=',', header=None,
                             names=names+['time_us']) for node in [1,2,3]]
        
        x = pd.concat(n_all)   
        
        x = x[x['numf'] < 120].reset_index(drop=True)
        
        steps = x['epochs']*x['dataset size']/x['batch']

        x['time_ms'] = x['time_us']/steps/1000
        
        self.df = x.drop(columns=['epochs', 'dataset size', 'time_us'])
        
    def plot(self, feat, label=None):
        dfs = [self.df[self.df['nodes']==node] for node in [1,2,3]]
        
        def org(df):
            return df[[feat, 'time_ms']].groupby(by=[feat]).mean().reset_index()
        
        scale = 2
        plt.figure(figsize=[scale*6.4, scale*4.8])
        
        colors = ['r','g','b']
        lines = [3,3,1]
        for j,marker in enumerate(['x','+','o']):
            _df = org(dfs[j])
            plt.scatter(_df[feat], _df['time_ms'], label='nodes: {}'.format(j+1), marker=marker,
                        s=100,
                        color=colors[j], 
                        linewidths=lines[j] 
                       )
            
        plt.legend(fontsize=20)
        plt.xlabel(feat if label == None else label, fontsize=20)
        plt.ylabel('Μέσος Χρόνος Εκτέλεσης (ms)', fontsize=20)
        plt.title(self.title, fontsize=20)
        plt.tight_layout()
    
    def _check(self, dim, fw):
        assert dim in [1,2]
        assert fw in ['tflow', 'ptorch']
        
    def all_names(self):
        return ['epochs', 'dataset size', 'numf', 'channels', 'batch', 'nodes']
    
    def dense_names(self):
        return ['epochs', 'dataset size', 'numf', 'batch', 'nodes', 'units']
    
    def the_fw(self, fw):
        assert fw in ['tflow', 'ptorch']
        dct = {'tflow':'TensorFlow', 'ptorch':'PyTorch'}
        
        return dct[fw]
        

In [ ]:
class Conv(Plot):
    def __init__(self, dim, fw):
        super()._check(dim, fw)
        
        self.file = 'conv{}d.{}'.format(dim, fw)
        super().__init__(self.file, super().all_names() + ['kernel', 'stride', 'filters'])
        
        self.title = 'Convolutional {}D - {}'.format(dim, self.the_fw(fw))

In [ ]:
class Pool(Plot):
    def __init__(self, typ, dim, fw):
        assert typ in ['avg', 'max']
        super()._check(dim, fw)
        
        self.file = '{}{}d.{}'.format(typ, dim, fw)
        super().__init__(self.file, super().all_names() + ['pool', 'stride'])
        
        dct = {'avg':'Average Pooling', 'max':'Max Pooling'}
        self.title = '{} {}D - {}'.format(dct[typ], dim, self.the_fw(fw))

In [ ]:
class Drop(Plot):
    def __init__(self, dim, fw):
        super()._check(dim, fw)
        self.file = 'drop{}d.{}'.format(dim, fw)
        
        super().__init__(self.file, super().all_names() + ['drop'])
        
        self.title = 'Dropout {}D - {}'.format(dim, self.the_fw(fw))

In [ ]:
class Rest(Plot):
    def __init__(self, typ, dim, fw):
        assert typ in ['norm', 'relu', 'tanh', 'flatten']
        super()._check(dim, fw)
        assert not ((typ=='flatten') & (fw=='tflow'))
        
        self.file = '{}{}d.{}'.format(typ, dim, fw)
        super().__init__(self.file, super().all_names())
        
        dct = {'norm':'Batch Normalization', 'relu':'ReLU', 'tanh':'Tanh', 'flatten':'Flatten'}
        self.title = '{} {}D - {}'.format(dct[typ], dim, self.the_fw(fw))

In [ ]:
class Dense(Plot):
    def __init__(self, fw, _final=False):
        assert fw in ['tflow', 'ptorch']
        final = _final if fw=='tflow' else True
        
        self.file = 'dense.{}'.format(fw) if final==False else 'final_dense.{}'.format(fw)
        super().__init__(self.file, super().dense_names())
        
        self.title = 'Fully Connected - {}'.format(self.the_fw(fw))

# Set TensorFlow or PyTorch

In [ ]:
fw = 'ptorch'

# Plot FC

In [ ]:
p = Dense(fw)

p.plot('numf', 'Πλήθος χαρακτηριστικών')
plt.savefig('{}_dense_numf'.format(fw), facecolor='white')

p.plot('batch', 'Μέγεθος τεμαχίων')
plt.savefig('{}_dense_batch'.format(fw), facecolor='white')

p.plot('units', 'Πλήθος μονάδων')
plt.savefig('{}_dense_units'.format(fw), facecolor='white')

# Set Dimension

In [ ]:
dim = 2

# Plot all the rest

In [ ]:
p = Rest('norm', dim, fw)

p.plot('numf', 'Πλήθος χαρακτηριστικών')
plt.savefig('{}_norm{}d_numf'.format(fw, dim), facecolor='white')

p.plot('channels', 'Πλήθος καναλιών εισόδου')
plt.savefig('{}_norm{}d_channels'.format(fw, dim), facecolor='white')

p.plot('batch', 'Μέγεθος τεμαχίων')
plt.savefig('{}_norm{}d_batch'.format(fw, dim), facecolor='white')

In [ ]:
p = Rest('relu', dim, fw)

p.plot('numf', 'Πλήθος χαρακτηριστικών')
plt.savefig('{}_relu{}d_numf'.format(fw, dim), facecolor='white')

p.plot('channels', 'Πλήθος καναλιών εισόδου')
plt.savefig('{}_relu{}d_channels'.format(fw, dim), facecolor='white')

p.plot('batch', 'Μέγεθος τεμαχίων')
plt.savefig('{}_relu{}d_batch'.format(fw, dim), facecolor='white')

In [ ]:
p = Rest('tanh', dim, fw)

p.plot('numf', 'Πλήθος χαρακτηριστικών')
plt.savefig('{}_tanh{}d_numf'.format(fw, dim), facecolor='white')

p.plot('channels', 'Πλήθος καναλιών εισόδου')
plt.savefig('{}_tanh{}d_channels'.format(fw, dim), facecolor='white')

p.plot('batch', 'Μέγεθος τεμαχίων')
plt.savefig('{}_tanh{}d_batch'.format(fw, dim), facecolor='white')

In [ ]:
p = Drop(dim, fw)

p.plot('numf', 'Πλήθος χαρακτηριστικών')
plt.savefig('{}_dropout{}d_numf'.format(fw, dim), facecolor='white')

p.plot('channels', 'Πλήθος καναλιών εισόδου')
plt.savefig('{}_dropout{}d_channels'.format(fw, dim), facecolor='white')

p.plot('batch', 'Μέγεθος τεμαχίων')
plt.savefig('{}_dropout{}d_batch'.format(fw, dim), facecolor='white')


p.plot('drop', 'Πιθανότητα εγκατάλειψης')
plt.savefig('{}_dropout{}d_kernel'.format(fw, dim), facecolor='white')

In [ ]:
p = Pool('avg', dim, fw)

p.plot('numf', 'Πλήθος χαρακτηριστικών')
plt.savefig('{}_avg{}d_numf'.format(fw, dim), facecolor='white')

p.plot('channels', 'Πλήθος καναλιών εισόδου')
plt.savefig('{}_avg{}d_channels'.format(fw, dim), facecolor='white')

p.plot('batch', 'Μέγεθος τεμαχίων')
plt.savefig('{}_avg{}d_batch'.format(fw, dim), facecolor='white')


p.plot('pool', 'Μέγεθος συγκέντρωσης')
plt.savefig('{}_avg{}d_kernel'.format(fw, dim), facecolor='white')

p.plot('stride', 'Βήμα ολίσθησης')
plt.savefig('{}_avg{}d_stride'.format(fw, dim), facecolor='white')

In [ ]:
p = Pool('max', dim, fw)

p.plot('numf', 'Πλήθος χαρακτηριστικών')
plt.savefig('{}_max{}d_numf'.format(fw, dim), facecolor='white')

p.plot('channels', 'Πλήθος καναλιών εισόδου')
plt.savefig('{}_max{}d_channels'.format(fw, dim), facecolor='white')

p.plot('batch', 'Μέγεθος τεμαχίων')
plt.savefig('{}_max{}d_batch'.format(fw, dim), facecolor='white')


p.plot('pool', 'Μέγεθος συγκέντρωσης')
plt.savefig('{}_max{}d_kernel'.format(fw, dim), facecolor='white')

p.plot('stride', 'Βήμα ολίσθησης')
plt.savefig('{}_max{}d_stride'.format(fw, dim), facecolor='white')

In [ ]:
p = Conv(dim,fw)
p.plot('numf', 'Πλήθος χαρακτηριστικών')
plt.savefig('{}_conv{}d_numf'.format(fw, dim), facecolor='white')

p.plot('channels', 'Πλήθος καναλιών εισόδου')
plt.savefig('{}_conv{}d_channels'.format(fw, dim), facecolor='white')

p.plot('batch', 'Μέγεθος τεμαχίων')
plt.savefig('{}_conv{}d_batch'.format(fw, dim), facecolor='white')


p.plot('filters', 'Πλήθος φίλτρων (Πλήθος καναλιών εξόδου)')
plt.savefig('{}_conv{}d_filters'.format(fw, dim), facecolor='white')

p.plot('kernel', 'Μέγεθος πυρήνα')
plt.savefig('{}_conv{}d_kernel'.format(fw, dim), facecolor='white')

p.plot('stride', 'Βήμα ολίσθησης')
plt.savefig('{}_conv{}d_stride'.format(fw, dim), facecolor='white')